# 01 Instalación de librerías y dependencias

In [ ]:
! pip install squarify

***IMPORTANTE***

Si corrió sin errores, ahora deben reiniciar el kernel (ir a menú "Entorno de ejecución" y escoger la opción "Reiniciar entorno de ejecución")

In [ ]:
# Importar librerias
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt
import squarify

# 02 Importación del archivo al entorno Colab y conversión a Dataframe (filas y columnas)

In [ ]:
# Debemos importar el archivo al entorno de python

from google.colab import files
uploaded = files.upload()

In [ ]:
# Debemos convertir el archivo a un dataframe (estructura de filas y columnas)
online = pd.read_csv('S03DS01 OnlineRetail.csv', encoding = "ISO-8859-1")
online.info()

# 03 Formato de columna fecha y tiempo

In [ ]:
from pandas.io.formats import info
#Debemos convertir la columna InvoiceDate de str a datatime
online['InvoiceDate'] = pd.to_datetime(online['InvoiceDate'])
online.info()

# 04 Solicitar un resumen del dataset

In [ ]:
print('{:,} filas; {:,} columnas'
      .format(online.shape[0], online.shape[1]))
print('{:,} transacciones sin Customer ID'
      .format(online[online.CustomerID.isnull()].shape[0]))
print('Fecha de Transacciones desde {} hasta {}'.format(online['InvoiceDate'].min(),
                                    online['InvoiceDate'].max()))

# 05 Análisis y eliminación de Missing Value

In [ ]:
#Debemos identificar los datos faltantes
#Todo las variables que figuren con información distinta a cero, nos indica cuantos datos faltantes tenemos

online.isnull().sum()

In [ ]:
# Drop NA values from online
online = online.dropna()

In [ ]:
online.isnull().sum()

In [ ]:
print('{:,} filas; {:,} columnas'
      .format(online.shape[0], online.shape[1]))
print('{:,} transacciones sin Customer ID'
      .format(online[online.CustomerID.isnull()].shape[0]))
print('Fecha de Transacciones desde {} hasta {}'.format(online['InvoiceDate'].min(),
                                    online['InvoiceDate'].max()))

# 06 Cálculo del monto facturado y fecha límite

In [ ]:
# --Group data by customerID--
# Create TotalSum column for online dataset
online['TotalSum'] = online['Quantity'] * online['UnitPrice']
online

In [ ]:
# Crear un fecha limite (superior de transacciones)
snapshot_date = online['InvoiceDate'].max() + timedelta(days=1)
print(snapshot_date)

# 07 Cálculo de Recencia, Frecuencia y Monto

In [ ]:
# Crear el GroupBy con la que calcularemos recencia, frecuencia y monto
data_process = online.groupby(['CustomerID']).agg({
        'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
        'InvoiceNo': 'count',
        'TotalSum': 'sum'})
data_process

In [ ]:
# Cambiar el nombre a las columnas creadas en el paso anterior
data_process.rename(columns={'InvoiceDate': 'Recency',
                         'InvoiceNo': 'Frequency',
                         'TotalSum': 'MonetaryValue'}, inplace=True)
data_process

In [ ]:
# Calcular el R grupo
# Crear una etiqueta de Recency
r_labels = range(4, 0, -1); f_labels = range(1, 5)
# Asiganar la etiquete de recencia por cuartil
r_groups = pd.qcut(data_process['Recency'], q=4, labels=r_labels)
r_groups

In [ ]:
# Asiganar la etiquete de frecuencia por cuartil
f_groups = pd.qcut(data_process['Frequency'], q=4, labels=f_labels)

In [ ]:
# Crear las nuevas columnas de Recencia (R) y Frecuencia (F)
data_process = data_process.assign(R = r_groups.values, F = f_groups.values)
data_process.head()

In [ ]:
# Crear la etiqueta para Monto
m_labels = range(1, 5)
# Asignar la etiqueta al cuartil correspondiente
m_groups = pd.qcut(data_process['MonetaryValue'], q=4, labels=m_labels)
# Crear la nueva columna M
data_process = data_process.assign(M = m_groups.values)
data_process.head()

In [ ]:
# Crear la columna RFM al concatenar la recencia, frecuencia y monto con la cual realizaremos el ranking
data_process['RFM_Segment_Concat'] = data_process.R.astype(str)+ data_process.F.astype(str) + data_process.M.astype(str)
rfm = data_process
rfm.head()

In [ ]:
# Contar el número unico de segmentos
rfm_count_unique = rfm.groupby('RFM_Segment_Concat')['RFM_Segment_Concat'].nunique()
print(rfm_count_unique.sum())

In [ ]:
# Cacluclar el Score de RFM
rfm['RFM_Score'] = rfm[['R','F','M']].sum(axis=1)
print(rfm['RFM_Score'].head())

In [ ]:
# Resumen del RFM Score
rfm['RFM_Score'].describe()

# 08 Asignación de tipo de segmento según score de RFM en español

In [ ]:
def rfm_level(df):
    if df['RFM_Score'] >= 9:
        return 'No los podemos perder'
    elif ((df['RFM_Score'] >= 8) and (df['RFM_Score'] < 9)):
        return 'Campeones'
    elif ((df['RFM_Score'] >= 7) and (df['RFM_Score'] < 8)):
        return 'Clientes fieles'
    elif ((df['RFM_Score'] >= 6) and (df['RFM_Score'] < 7)):
        return 'Con Potencial'
    elif ((df['RFM_Score'] >= 5) and (df['RFM_Score'] < 6)):
        return 'Promesas'
    elif ((df['RFM_Score'] >= 4) and (df['RFM_Score'] < 5)):
        return 'Necesitan atención'
    else:
        return 'Requieren Activación'
# Create a new variable RFM_Level
rfm['RFM_Level'] = rfm.apply(rfm_level, axis=1)
# Print the header with top 5 rows to the console
rfm.head()

In [ ]:
# Calcular los promedios y conteos de segmento por cada nivel de RFM
rfm_level_agg = rfm.groupby('RFM_Level').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': ['mean', 'count']
}).round(1)
# Print the aggregated dataset
print(rfm_level_agg)

In [ ]:
rfm_level_agg

In [ ]:
rfm_level_agg.columns = ['RecencyMean','FrequencyMean','MonetaryMean', 'Count']

# Gráfico de los segmento por nivel de RFM
fig = plt.gcf()
ax = fig.add_subplot()
fig.set_size_inches(16, 9)
squarify.plot(sizes=rfm_level_agg['Count'], 
              label=['Campeones',
                     'Clientes fieles',
                     'Con Potencial',
                     'Necesitan atención',
                     'No los podemos perder', 
                     'Promesas', 
                     'Requieren Activación'], alpha=.6 )

plt.title("RFM Segments",fontsize=18,fontweight="bold")
plt.axis('off')
plt.show()

In [ ]:
# Obtener un archivo completo de RFM que podamos posteriormente análisar
rfm_completo = rfm.merge(online, left_on='CustomerID', right_on='CustomerID')
rfm_completo

In [ ]:
#exportar el archivo a "colab"
rfm_completo.to_csv(r'S03 Output RFM ES.csv')

# 09 Asignación de tipo de segmento según score de RFM en ingles

In [ ]:
# Define rfm_level function EG
def rfm_level(df):
    if df['RFM_Score'] >= 9:
        return 'Can\'t Loose Them'
    elif ((df['RFM_Score'] >= 8) and (df['RFM_Score'] < 9)):
        return 'Champions'
    elif ((df['RFM_Score'] >= 7) and (df['RFM_Score'] < 8)):
        return 'Loyal'
    elif ((df['RFM_Score'] >= 6) and (df['RFM_Score'] < 7)):
        return 'Potential'
    elif ((df['RFM_Score'] >= 5) and (df['RFM_Score'] < 6)):
        return 'Promising'
    elif ((df['RFM_Score'] >= 4) and (df['RFM_Score'] < 5)):
        return 'Needs Attention'
    else:
        return 'Require Activation'
# Create a new variable RFM_Level
rfm['RFM_Level'] = rfm.apply(rfm_level, axis=1)
# Print the header with top 5 rows to the console
rfm.head()

In [ ]:
# Calcular los promedios y conteos de segmento por cada nivel de RFM
rfm_level_agg = rfm.groupby('RFM_Level').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': ['mean', 'count']
}).round(1)
# Print the aggregated dataset
print(rfm_level_agg)

In [ ]:
rfm_level_agg

In [ ]:
rfm_level_agg.columns = rfm_level_agg.columns.droplevel()
rfm_level_agg.columns = ['RecencyMean','FrequencyMean','MonetaryMean', 'Count']

#Create our plot and resize it.
fig = plt.gcf()
ax = fig.add_subplot()
fig.set_size_inches(16, 9)
squarify.plot(sizes=rfm_level_agg['Count'], 
              label=['Can\'t Loose Them',
                     'Champions',
                     'Loyal',
                     'Needs Attention',
                     'Potential', 
                     'Promising', 
                     'Require Activation'], alpha=.6 )

plt.title("RFM Segments",fontsize=18,fontweight="bold")
plt.axis('off')
plt.show()

In [ ]:
# Obtener un archivo completo de RFM que podamos posteriormente análisar
rfm_completo = rfm.merge(online, left_on='CustomerID', right_on='CustomerID')
rfm_completo

In [ ]:
#exportar el archivo a "colab"
rfm_completo.to_csv(r'S03 Output RFM EN.csv')